In [40]:
// Cell 1: Setup Dependencies

@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.7.3")
@file:DependsOn("com.opencsv:opencsv:5.5")

import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.config.solver.SolverConfig
import java.time.LocalDateTime
import java.time.LocalTime
import java.time.DayOfWeek
import java.time.Duration
import kotlin.math.sqrt
import com.opencsv.CSVWriter
import java.io.FileWriter

In [41]:
// Cell 2: Define Domain Models

data class Location(
    val name: String,
    val latitude: Double,
    val longitude: Double
) {
    fun distanceTo(other: Location): Double {
        val latDiff = latitude - other.latitude
        val lonDiff = longitude - other.longitude
        return sqrt(latDiff * latDiff + lonDiff * lonDiff)
    }
}

data class Skill(val name: String)

data class Shift(
    val name: String,
    val startTime: LocalTime,
    val endTime: LocalTime
)

data class Agent(
    @PlanningId val id: Long,
    val name: String,
    val skills: List<Skill>,
    val homeLocation: Location
)

data class Meeting(
    @PlanningId val id: Long,
    val client: String,
    val location: Location,
    val requiredSkills: List<Skill>,
    val startDateTime: LocalDateTime,
    val endDateTime: LocalDateTime,
    val requiredAgents: Int,
    val shift: Shift
) {
    fun duration(): Duration = Duration.between(startDateTime, endDateTime)
}

@PlanningEntity
data class AgentAssignment(
    @PlanningId val id: Long,
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"])
    var agent: Agent? = null,
    val meeting: Meeting
)

@PlanningSolution
data class Schedule(
    @ValueRangeProvider(id = "agentRange")
    @ProblemFactCollectionProperty
    val agents: List<Agent>,
    @ProblemFactCollectionProperty
    val meetings: List<Meeting>,
    @PlanningEntityCollectionProperty
    val assignments: List<AgentAssignment>,
    @ProblemFactCollectionProperty
    val skills: List<Skill>,
    @ProblemFactCollectionProperty
    val shifts: List<Shift>
) {
    @PlanningScore
    var score: HardMediumSoftLongScore? = null

    // No-arg constructor for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList(), emptyList(), emptyList())
}


In [42]:
// Cell 3: Generate Fake Data

val numberOfAgents = 10
val numberOfMeetings = numberOfAgents * 5 * 7 // Approximately 5 meetings per agent per day for a week

val skills = listOf(
    Skill("Fire Certification"),
    Skill("Bodyguard Certification"),
    Skill("First Aid Certification"),
    Skill("Fighting Certification")
)

val shifts = listOf(
    Shift("Morning", LocalTime.of(8, 0), LocalTime.of(16, 0)),
    Shift("Afternoon", LocalTime.of(11, 0), LocalTime.of(19, 0)),
    Shift("Night", LocalTime.of(20, 0), LocalTime.of(4, 0)) // Ends at 4 am next day
)

fun generateLocations(count: Int): List<Location> {
    val genevaLatitude = 46.2044
    val genevaLongitude = 6.1432
    return (1..count).map { id ->
        Location(
            name = "Location_$id",
            latitude = genevaLatitude + (0.02 * (Math.random() - 0.5)),
            longitude = genevaLongitude + (0.02 * (Math.random() - 0.5))
        )
    }
}

fun generateAgents(count: Int, skills: List<Skill>, homeLocation: Location): List<Agent> {
    return (1..count).map { id ->
        Agent(
            id = id.toLong(),
            name = "Agent_$id",
            skills = skills.shuffled().take((1..skills.size).random()),
            homeLocation = homeLocation
        )
    }
}

fun generateMeetings(
    count: Int,
    skills: List<Skill>,
    locations: List<Location>,
    shifts: List<Shift>
): List<Meeting> {
    val startDate = LocalDateTime.now().withHour(0).withMinute(0).withSecond(0).withNano(0)
    return (1..count).map { id ->
        val requiredSkillCount = (1..3).random()
        val requiredSkills = skills.shuffled().take(requiredSkillCount)
        val location = locations.random()
        val shift = shifts.random()
        val dayOffset = (0..6).random() // 0-6 for a week
        val startDateTime = startDate.plusDays(dayOffset.toLong())
            .withHour(shift.startTime.hour)
            .withMinute(shift.startTime.minute)
            .plusHours((0..7).random().toLong())
        val duration = Duration.ofHours((1..3).random().toLong())
        val endDateTime = startDateTime.plus(duration)
        val isNightShift = shift.name == "Night"
        val requiredAgents = if (isNightShift) (1..2).random() else 1

        val meetingSkills = if (isNightShift) {
            val fightingSkill = skills.find { it.name == "Fighting Certification" }!!
            if (requiredSkills.contains(fightingSkill)) requiredSkills else requiredSkills + fightingSkill
        } else {
            requiredSkills
        }

        Meeting(
            id = id.toLong(),
            client = "Client_$id",
            location = location,
            requiredSkills = meetingSkills,
            startDateTime = startDateTime,
            endDateTime = endDateTime,
            requiredAgents = requiredAgents,
            shift = shift
        )
    }
}

// Generate data
val homeLocation = Location("Home_Geneva", 46.2044, 6.1432)
val meetingLocations = generateLocations(20)
val agents = generateAgents(numberOfAgents, skills, homeLocation)
val meetings = generateMeetings(numberOfMeetings, skills, meetingLocations, shifts)

// Generate AgentAssignments
var assignmentId = 1L
val assignments = meetings.flatMap { meeting ->
    (1..meeting.requiredAgents).map {
        AgentAssignment(
            id = assignmentId++,
            agent = null,
            meeting = meeting
        )
    }
}

// Create the initial schedule
val initialSchedule = Schedule(agents, meetings, assignments, skills, shifts)


In [43]:
// Cell 4: Define Constraints

import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import java.time.Duration

class SecurityConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            agentConflict(constraintFactory),
            skillRequirements(constraintFactory),
            requiredAgentsPerMeeting(constraintFactory),
            avoidConsecutiveNightShifts(constraintFactory),
            respectBreakTime(constraintFactory),
            minimizeTravelTime(constraintFactory),
            balanceWorkload(constraintFactory)
        )
    }

    /**
     * Constraint: An agent cannot be assigned to overlapping meetings.
     */
    fun agentConflict(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.overlapping(
                    { it.meeting.startDateTime },
                    { it.meeting.endDateTime }
                )
            )
            .penalizeLong(HardMediumSoftLongScore.ONE_HARD,
                { assignment1, assignment2 ->
                    // Penalize based on the total overlapping minutes
                    Duration.between(assignment1.meeting.startDateTime, assignment1.meeting.endDateTime).toMinutes() +
                            Duration.between(assignment2.meeting.startDateTime, assignment2.meeting.endDateTime).toMinutes()
                })
            .asConstraint("Agent conflict")
    }

    /**
     * Constraint: An agent must possess all required skills for a meeting they're assigned to.
     */
    fun skillRequirements(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(AgentAssignment::class.java)
            .filter { assignment ->
                // Check if the agent has all the required skills for the meeting
                assignment.agent?.skills?.containsAll(assignment.meeting.requiredSkills) != true
            }
            .penalizeLong(HardMediumSoftLongScore.ONE_HARD,
                { assignment -> assignment.meeting.requiredSkills.size.toLong() })
            .asConstraint("Skill requirements")
    }

    /**
     * Constraint: Each meeting must have exactly the required number of agents assigned.
     */
    fun requiredAgentsPerMeeting(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Meeting::class.java)
            .join(AgentAssignment::class.java,
                Joiners.equal(Meeting::id, { it.meeting.id })
            )
            .groupBy({ meeting: Meeting -> meeting },
                ConstraintCollectors.countLong())
            .filter { meeting, count -> count != meeting.requiredAgents }
            .penalizeLong(HardMediumSoftLongScore.ONE_HARD,
                { meeting, count -> Math.abs(count - meeting.requiredAgents).toLong() })
            .asConstraint("Required agents per meeting")
    }

    /**
     * Constraint: Prevent agents from having consecutive night shifts.
     */
    fun avoidConsecutiveNightShifts(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.equal({ it.meeting.shift.name }, { "Night" }),
                Joiners.equal({ it.meeting.startDateTime.toLocalDate() },
                    { it.meeting.startDateTime.toLocalDate().plusDays(1) })
            )
            .penalizeLong(HardMediumSoftLongScore.ONE_MEDIUM, 1L)
            .asConstraint("Avoid consecutive night shifts")
    }

    /**
     * Constraint: Ensure at least 8 hours of break between consecutive meetings for an agent.
     */
    fun respectBreakTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.lessThan(
                    { it.meeting.endDateTime },
                    { it.meeting.startDateTime }
                )
            )
            .filter { assignment1, assignment2 ->
                Duration.between(assignment1.meeting.endDateTime, assignment2.meeting.startDateTime).toHours() < 8
            }
            .penalizeLong(HardMediumSoftLongScore.ONE_SOFT,
                { assignment1, assignment2 ->
                    // Penalize the shortage in hours
                    (8 - Duration.between(assignment1.meeting.endDateTime, assignment2.meeting.startDateTime).toHours()).toLong()
                })
            .asConstraint("Respect break time")
    }

    /**
     * Constraint: Minimize the travel time between consecutive meetings for an agent.
     */
    fun minimizeTravelTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.lessThan(
                    { it.meeting.endDateTime },
                    { it.meeting.startDateTime }
                )
            )
            .penalizeLong(HardMediumSoftLongScore.ONE_SOFT,
                { assignment1, assignment2 ->
                    // Penalize based on the distance between meeting locations (assuming distanceTo returns kilometers)
                    (assignment1.meeting.location.distanceTo(assignment2.meeting.location) * 1000).toLong()
                })
            .asConstraint("Minimize travel time")
    }

    /**
     * Constraint: Balance the workload by ensuring agents do not exceed 40 hours per week.
     */
    fun balanceWorkload(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Agent::class.java)
            .join(AgentAssignment::class.java,
                Joiners.equal({ agent: Agent -> agent }, AgentAssignment::agent)
            )
            .groupBy({ agent: Agent, _: AgentAssignment -> agent },
                ConstraintCollectors.sumLong { _: Agent, assignment: AgentAssignment -> assignment.meeting.duration().toHours() })
            .penalizeLong(HardMediumSoftLongScore.ONE_SOFT,
                { _: Agent, totalHours: Long -> (totalHours - 40).coerceAtLeast(0) })
            .asConstraint("Balance workload")
    }
}


Line_41.jupyter.kts (69:22 - 53) Type mismatch: inferred type is (Line_39_jupyter.Meeting) -> Line_39_jupyter.Meeting but BiConstraintCollector<Line_39_jupyter.Meeting!, Line_39_jupyter.AgentAssignment!, TypeVariable(ResultContainerA_)!, TypeVariable(ResultA_)!>! was expected
Line_41.jupyter.kts (70:17 - 49) Type mismatch: inferred type is UniConstraintCollector<(???..???), *, Long!>! but BiConstraintCollector<Line_39_jupyter.Meeting!, Line_39_jupyter.AgentAssignment!, TypeVariable(ResultContainerB_)!, TypeVariable(ResultB_)!>! was expected
Line_41.jupyter.kts (71:23 - 30) Cannot infer a type for this parameter. Please specify it explicitly.
Line_41.jupyter.kts (71:32 - 37) Cannot infer a type for this parameter. Please specify it explicitly.
Line_41.jupyter.kts (73:19 - 26) Cannot infer a type for this parameter. Please specify it explicitly.
Line_41.jupyter.kts (73:28 - 33) Cannot infer a type for this parameter. Please specify it explicitly.
Line_41.jupyter.kts (84:25 - 30) Overload